<a href="https://colab.research.google.com/github/callysthenes/reinforcement_learning/blob/main/constrained_optimization_student_version.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import numpy as np
from scipy.optimize import minimize

In [9]:
# Define the objective function
def objective(x):
    # note we are gonna minimize the objective function, but we wanna maximize revenue
    return -(10*x[0]+20*x[1]+30*x[2])

In [10]:
# Define the constraint functions: 

# From scipy documentation:

# Equality constraint means that the constraint function result is to be zero 
# whereas inequality means that it is to be non-negative (>=0)

def capacity_constraint(x):
    # inequality
    return 25 - (2*x[0]+3*x[1] +4*x[2])

def demand_constraint(x):
    # equality
    return x[0] +x[1]+x[2] -10

# Define the constraints
cons = ({'type': 'ineq', 'fun': capacity_constraint},
        {'type': 'eq', 'fun': demand_constraint})

In [11]:
# Define the initial guess
x0 = np.array([1,1,1])

In [12]:
# Define the bounds for the variables
bounds = ((0,None), (0,None), (0,None))

# Solve with SLSQP (Sequential Least Squares Programming )

In [13]:
# Solve the optimization problem
result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)
#very traditional algorithm 


# Print the results
print('Optimal solution:', result.x)
print('Optimal objective value:', -result.fun)

Optimal solution: [5.83333333 3.33333333 0.83333333]
Optimal objective value: 149.9999999999895


Note the constraints can be slightly negative, this is expected with numerical solvers

In [14]:
x = result.x
-objective(x), capacity_constraint(x), demand_constraint(x)

(149.9999999999895, 1.0444978215673473e-12, 0.0)

In [17]:
result.message

'Optimization terminated successfully'

In [18]:
result.status

0

In [19]:
result.nfev

8

In [20]:
result.nit

2

## Are there better solutions?

In [22]:
x=[10,0,0]
-objective(x),capacity_constraint(x), demand_constraint(x)

(100, 5, 0)

In [23]:
x=[7,1,2]
-objective(x),capacity_constraint(x), demand_constraint(x)

(150, 0, 0)

In [24]:
x=[6,1,3]
-objective(x),capacity_constraint(x), demand_constraint(x)

(170, -2, 0)

In [25]:
x=[6,3,1]
-objective(x),capacity_constraint(x), demand_constraint(x)

(150, 0, 0)

This solution is equivalent to the 7,1,2 combination

Try converting the demand equality to an inequality, as the stock can be larger than the demand

In [26]:
x0 = (1,1,1)
cons = ({'type': 'ineq', 'fun': capacity_constraint},
        {'type': 'ineq', 'fun': demand_constraint})

result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=cons)

# Print the results
print('Optimal solution:', result.x)
print('Optimal objective value:', result.fun)
print(result.message, result.status, result.success, result.nfev, result.nit)

x = result.x
-objective(x), capacity_constraint(x), demand_constraint(x)

Optimal solution: [5.83333333 3.33333333 0.83333333]
Optimal objective value: -150.00000000040583
Optimization terminated successfully 0 True 8 2


(150.00000000040583, -5.7056581681536045e-11, 1.6477486042276723e-11)

# COBYLA (Constrained Optimization BY Linear Approximation )


Note cobyla does not accept equality constraint, neither bounds. So we need to convert demand equality to an inequality, and rewrite bounds as contraints (x[0] > 0, for instance)

Exam question: what is wrong with this code. for example, this constraint is not well adjusted

In [29]:
x0 = [1, 1, 1]
cobyla_cons = (
    {'type': 'ineq', 'fun': capacity_constraint},
    {'type': 'ineq', 'fun': demand_constraint},
    {'type': 'ineq', 'fun': lambda x: x[0]}, 
    {'type': 'ineq', 'fun': lambda x: x[1]}, 
    {'type': 'ineq', 'fun': lambda x: x[2]}, 
    )

result = minimize(objective, x0, method='cobyla', constraints=cobyla_cons)
print('Optimal solution:', result.x)
print('Optimal objective value:', result.fun)
print(result.message, result.status, result.success, result.nfev)

x = result.x
-objective(x), capacity_constraint(x), demand_constraint(x)

Optimal solution: [5.36125542 4.27748915 0.36125542]
Optimal objective value: -150.00000000000003
Optimization terminated successfully. 1 True 48


(150.00000000000003, 0.0, -1.7763568394002505e-15)

In [30]:
x0 = [6, 1, 1]
cobyla_cons = (
    {'type': 'ineq', 'fun': capacity_constraint},
    {'type': 'ineq', 'fun': demand_constraint},
    {'type': 'ineq', 'fun': lambda x: x[0]}, 
    {'type': 'ineq', 'fun': lambda x: x[1]}, 
    {'type': 'ineq', 'fun': lambda x: x[2]}, 
    )

result = minimize(objective, x0, method='cobyla', constraints=cobyla_cons)
print('Optimal solution:', result.x)
print('Optimal objective value:', result.fun)
print(result.message, result.status, result.success, result.nfev)

x = result.x
-objective(x), capacity_constraint(x), demand_constraint(x)

Optimal solution: [6.5 2.  1.5]
Optimal objective value: -150.0
Optimization terminated successfully. 1 True 24


(150.0, 0.0, 0.0)

Is this solution valid? this is a valid solution because both constrains are zero

In [31]:
x = [7,2,2]
-objective(x), capacity_constraint(x), demand_constraint(x)

(170, -3, 1)

This solution is not valid because the capacity constraint is minus three, not acceptable. It could be acceptable if -0.000001 but not -3